In [ ]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

import pickle
import joblib
import re
from tqdm import tqdm_notebook
import os

In [ ]:
# DTM 문서 단어 행렬
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer()

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer    
tfvector = TfidfTransformer(smooth_idf=False)

In [ ]:
vectorizer=TfidfVectorizer(ngram_range=(1, 3),
    min_df=2,       
    max_features=10000,
    sublinear_tf=True,
    lowercase=False,
    use_idf=True)

In [ ]:
#function ClickConnect() {var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); buttons.forEach(function(btn) { btn.click(); }); console.log("1분마다 자동 재연결"); document.querySelector("colab-toolbar-button#connect").click(); } setInterval(ClickConnect,1000*60);


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
ls

gdrive/  sample_data/


In [ ]:
cd gdrive/My\ Drive/kb_data

/content/gdrive/My Drive/kb_data


In [ ]:
prepared_train = pd.read_csv('prepared_train.csv', encoding='utf-8')

In [ ]:
del prepared_train['Unnamed: 0']

In [ ]:
%%time
vec_train = vectorizer.fit_transform(prepared_train['orgin'])

CPU times: user 49.6 s, sys: 1.52 s, total: 51.1 s
Wall time: 51.1 s


In [ ]:
svm_model = joblib.load('final_model/svm_model.pkl')
lgbm_model = joblib.load('final_model/lgbm_model.pkl')

In [ ]:
X_origin = vec_train
Y_label = prepared_train['smishing']

In [ ]:
# train - test 분류
# 인덱싱 문제로 순서대로 train / test split
%%time
X_train, X_test, Y_train, Y_test = train_test_split(X_origin,
                                                    Y_label,
                                                    test_size = 0.2,
                                                    shuffle = False)
                                                    # random_state = 1825)

CPU times: user 57.7 ms, sys: 0 ns, total: 57.7 ms
Wall time: 58.8 ms


## 단일모델: SVM

In [ ]:
svm_submission = pd.DataFrame(data=np.zeros((X_test.shape[0],2)))
svm_submission.index = Y_test.index 
svm_submission += svm_model.predict_proba(X_test)
svm_submission.index.name = 'id'
svm_submission.columns = ['normal', 'smishing']


### 예측값 산정

In [ ]:
svm_submission['pred'] = 0
for idx in svm_submission.index:
    if (svm_submission['normal'][idx] < svm_submission['smishing'][idx]):
        svm_submission['pred'][idx]= 1

In [ ]:
svm_submission.to_csv('final_model/submissions/svm_submission.csv')

### classification report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
result = classification_report(Y_test, svm_submission['pred'], target_names=['normal','smishing'])

In [ ]:
print(result)

## 직렬모델: SVM + LGBM 
(SVM의 예측확률이 90% 넘어가는 것 -> LGBM의 확률과 더한 후 확률로 판정)

In [ ]:
check_list = [0.1, 0.2, 0.3, 0.4]
result_dict = dict()

for prob in check_list:
    svmNlgbm_submission = pd.DataFrame(data=np.zeros((X_test.shape[0],2)))
    svmNlgbm_submission.index = Y_test.index
    svmNlgbm_submission.columns = ['normal', 'smishing']
    svmNlgbm_submission += svm_model.predict_proba(X_test)
    svmNlgbm_submission.index.name = 'id'
    svmNlgbm_submission['pred'] = 99

    for test_idx, idx in tqdm_notebook(enumerate(svmNlgbm_submission.index)):
        # 'smishing'에 대한 예측 확률 (svm 대부분 99.9 이상으로 갈림 -> 0.1 ~ 0.9 사이면 확신이 없는 것임)
        if ( prob < svm_model.predict_proba(X_test[test_idx])[0][1] < 1 - prob):
            smishing_prob = (svm_model.predict_proba(X_test[test_idx])[0][1] + lgbm_model.predict_proba(X_test[test_idx])[0][1]) / 2
            svmNlgbm_submission['smishing'][idx] = smishing_prob
            svmNlgbm_submission['normal'][idx] = 1 - smishing_prob
            if (smishing_prob > 0.5):
                svmNlgbm_submission['pred'][idx] = 1
            else:
                svmNlgbm_submission['pred'][idx] = 0
        else:
            svmNlgbm_submission['normal'][idx] = svm_model.predict_proba(X_test[test_idx])[0][0]
            svmNlgbm_submission['smishing'][idx] = svm_model.predict_proba(X_test[test_idx])[0][1]
            # smishing 확률 더 높음
            if (svm_model.predict_proba(X_test[test_idx])[0][1] > svm_model.predict_proba(X_test[test_idx])[0][0]):
                svmNlgbm_submission['pred'][idx] = 1
            # normal 확률 더 높음
            else:
                svmNlgbm_submission['pred'][idx] = 0

    svmNlgbm_submission.index = Y_test.index 
    svmNlgbm_submission.to_csv('final_model/submissions/svmNlgbm_submission' + str(prob) + 'N' + str(1-prob) + '.csv')
    try:
        result = classification_report(Y_test, svmNlgbm_submission['pred'], target_names=['normal','smishing'])
        result_dict[idx] = result
        print()
        print(result)
    except:
        pass




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to b



              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      0.99      7984

    accuracy                           1.00     59189
   macro avg       1.00      0.99      1.00     59189
weighted avg       1.00      1.00      1.00     59189





              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      1.00      7984

    accuracy                           1.00     59189
   macro avg       1.00      1.00      1.00     59189
weighted avg       1.00      1.00      1.00     59189





              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      0.99      7984

    accuracy                           1.00     59189
   macro avg       1.00      1.00      1.00     59189
weighted avg       1.00      1.00      1.00     59189





              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      0.99      7984

    accuracy                           1.00     59189
   macro avg       1.00      0.99      1.00     59189
weighted avg       1.00      1.00      1.00     59189



In [ ]:
## 보관용

# svmNlgbm_submission = pd.DataFrame(data=np.zeros((X_test.shape[0],2)))
# svmNlgbm_submission.index = Y_test.index 
# svmNlgbm_submission += svm_model.predict_proba(X_test)
# svmNlgbm_submission.index.name = 'id'
# svmNlgbm_submission.columns = ['normal', 'smishing']
# svmNlgbm_submission['pred'] = 0

# result_dict = dict()

# for idx in tqdm(range(X_test.shape[0])):
#     # 'smishing'에 대한 예측 확률 (svm 대부분 99.9 이상으로 갈림 -> 0.1 ~ 0.9 사이면 확신이 없는 것임)
#     if (0.3 < svm_model.predict_proba(X_test[idx])[0][1] < 0.7):
#         smishing_prob = (svm_model.predict_proba(X_test[idx])[0][1] + lgbm_model.predict_proba(X_test[idx])[0][1]) / 2
#         svmNlgbm_submission['smishing'][idx] = smishing_prob
#         svmNlgbm_submission['normal'][idx] = 1 - smishing_prob
#         if (smishing_prob > 0.5):
#             print("애매한 경우 확률: ", smishing_prob)
#             svmNlgbm_submission['pred'][idx] = 1
#         else:
#             svmNlgbm_submission['pred'][idx] = 0
#     else:
#         svmNlgbm_submission['normal'][idx] = svm_model.predict_proba(X_test[idx])[0][0]
#         svmNlgbm_submission['smishing'][idx] = svm_model.predict_proba(X_test[idx])[0][1]
#         # smishing 확률 더 높음
#         if (svm_model.predict_proba(X_test[idx])[0][1] > svm_model.predict_proba(X_test[idx])[0][0]):
#             svmNlgbm_submission['pred'][idx] = 1
#         # normal 확률 더 높음
#         else:
#             svmNlgbm_submission['pred'][idx] = 0

# svmNlgbm_submission.index = Y_test.index 
# svmNlgbm_submission.to_csv('final_model/submissions/svmNlgbm_submission3070.csv')
# result = classification_report(Y_test, svmNlgbm_submission['pred'], target_names=['normal','smishing'])

# print(result)

# 최종결과
: 정확도 및 F1-score가 SVM의 분류 확률이 2:8로 나눠진 것부터 LGBM 보완하면 더 성능이 높아짐<br>
(기존: 정확도 0.9983 / F1 socre: 0.9939)<br>
(LGBM 직렬 처리 후 : 정확도 0.9987 / F1 score 0.9954)

In [ ]:
check_list = [0.1, 0.2, 0.3, 0.4]
Y_label = prepared_train['smishing']
Y_train, Y_test = train_test_split(Y_label,test_size = 0.2,shuffle = False)

svm = pd.read_csv('final_model/submissions/svm_submission.csv')
print("svm 단독 모델: ", accuracy_score(Y_test, svm['pred']))
print(f1_score(Y_test, svm['pred'], labels = ['normal','smishing'], pos_label = 1))
print()
print()

result_dict = dict()
for prob in check_list:
    sub = pd.read_csv('final_model/submissions/svmNlgbm_submission' + str(prob) + 'N' + str(1-prob) + '.csv')
    print(accuracy_score(Y_test, sub['pred']))
    print(f1_score(Y_test, sub['pred'], labels = ['normal','smishing'], pos_label = 1))
    result = classification_report(Y_test, sub['pred'], target_names=['normal','smishing'])
    print(result)
    result_dict[prob] = result
    print()
    print()
   


svm 단독 모델:  0.9983780770075521
0.9939577039274924


0.9986315024751221
0.9949014917857367
              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      0.99      7984

    accuracy                           1.00     59189
   macro avg       1.00      0.99      1.00     59189
weighted avg       1.00      1.00      1.00     59189



0.9987835577556641
0.9954711284438295
              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      1.00      7984

    accuracy                           1.00     59189
   macro avg       1.00      1.00      1.00     59189
weighted avg       1.00      1.00      1.00     59189



0.9986483975062934
0.994966654083302
              precision    recall  f1-score   support

      normal       1.00      1.00      1.00     51205
    smishing       1.00      0.99      0.99      7984

    accuracy 

In [ ]:
result_dict

{0.1: '              precision    recall  f1-score   support\n\n      normal       1.00      1.00      1.00     51205\n    smishing       1.00      0.99      0.99      7984\n\n    accuracy                           1.00     59189\n   macro avg       1.00      0.99      1.00     59189\nweighted avg       1.00      1.00      1.00     59189\n',
 0.2: '              precision    recall  f1-score   support\n\n      normal       1.00      1.00      1.00     51205\n    smishing       1.00      0.99      1.00      7984\n\n    accuracy                           1.00     59189\n   macro avg       1.00      1.00      1.00     59189\nweighted avg       1.00      1.00      1.00     59189\n',
 0.3: '              precision    recall  f1-score   support\n\n      normal       1.00      1.00      1.00     51205\n    smishing       1.00      0.99      0.99      7984\n\n    accuracy                           1.00     59189\n   macro avg       1.00      1.00      1.00     59189\nweighted avg       1.00   